<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigdataII_06_pca_part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# いつものDriveマウント

#from google.colab import drive
#drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/Lecture_BigData'

## 主成分分析（PCA） 演習

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Matplotlib文字化け対策

In [ ]:
x = range(6)
y = np.linspace(0,5,6)

plt.plot(x, y)
plt.title("テスト")
plt.show()

In [ ]:
# 「IPA」フォントをインストール
!apt-get -y install fonts-ipafont-gothic
#!rm /root/.cache/matplotlib/fontlist-v300.json # キャッシュを削除

!pip install japanize-matplotlib
import japanize_matplotlib #matplotlib日本語化
import seaborn as sns
sns.set(font="IPAexGothic") #日本語フォント設定

In [ ]:
# タイトル確認
plt.plot(x, y)
plt.title("テスト")
plt.show()

## 基本的な使い方

データ生成

In [ ]:
# ランダムにデータを生成
here

# 分散させるために2x2の行列を掛ける
here

In [ ]:
plt.scatter(X[:, 0], X[:, 1])
plt.axis('equal'); # 軸の比率を同じにする

PCA

主成分と共分散を取得する

In [ ]:
from sklearn.decomposition import PCA

here

In [ ]:
print('主成分\n', pca.components_)
print('平均\n', pca.mean_)
print('分散\n', pca.explained_variance_)
print('共分散\n', pca.get_covariance())

平均と共分散を直接計算して、比較してみる。

In [ ]:
mu = 

z = 

cv = 

print('平均\n', mu)
print('共分散\n', cv)

補足：

共分散行列の固有値と固有ベクトルを計算する。
共分散行列の固有ベクトルは主成分と一致する。

※データによっては180度反転してる場合もある

In [ ]:
W, v = np.linalg.eig(cv)

print('固有値\n', W)
print('固有ベクトル\n', v)

ベクトルを描画する関数

> 参考：矢印を描画する　https://qiita.com/haru1843/items/e85fe45163cd9763023c

> 参考：for loopで便利な zip, enumerate　https://python.civic-apps.com/zip-enumerate/

In [ ]:
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(
                arrowstyle='->',
                linewidth=2,
                shrinkA=0, shrinkB=0,
                color="red")
    ax.annotate('', v1, v0, arrowprops=arrowprops)

オリジナルデータのプロットと、主成分を軸にしたプロット

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

# オリジナルデータをプロット
ax[0].scatter(X[:, 0], X[:, 1], alpha=0.5)
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    draw_vector(pca.mean_, pca.mean_ + v, ax=ax[0])
ax[0].axis('equal');
ax[0].set(
          xlabel='x', ylabel='y',
          title='input')

# 平均 0、分散 1にする : whitening
pca = PCA(n_components=2, whiten=True)
pca.fit(X)

# 主成分（PC1, PC2）空間へのプロット
# pca.transform() : 主成分を主軸とするような変換を行う
here

ax[1].scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.5)
draw_vector([0, 0], [0, 3], ax=ax[1])
draw_vector([0, 0], [3, 0], ax=ax[1])
ax[1].axis('equal')
ax[1].set(
          xlabel='component 1', ylabel='component 2',
          title='principal components',
          xlim=(-5, 5), ylim=(-3, 3.1));

次元削減

In [ ]:
# 2次元 ⇒ 1次元
pca = PCA(n_components=1)
pca.fit(X)
X_pca = pca.transform(X)
print("original shape:   ", X.shape)
print("transformed shape:", X_pca.shape)

In [ ]:
# PC1軸への射影プロット
X_new = pca.inverse_transform(X_pca)
plt.scatter(X[:, 0], X[:, 1], alpha=0.3)
plt.scatter(X_new[:, 0], X_new[:, 1], color='red')
plt.axis('equal');

## Wine データセット

178行のワインサンプルと、それらの科学的性質を表す13列の特徴量で構成されているWineデータセットを読み込む。

In [ ]:
import pandas as pd
from sklearn.datasets import load_wine
import matplotlib.pyplot as plt
import numpy as np

wine = load_wine()  # Wineデータセットの読み込み
df_wine = pd.DataFrame(wine.data, columns=wine.feature_names)
df_wine['class'] = wine.target
df_wine

データの標準化（平均０、分散１）

In [ ]:
from sklearn.preprocessing import StandardScaler

X = df_wine.iloc[:, :-1].values  # classカラム以外を取得
y = df_wine.iloc[:, -1].values  # classカラムを取得

# 標準化
sc = StandardScaler()
X_std = sc.fit_transform(X)

X_std

PCA実行

In [ ]:
# PCAライブラリをインポート
from sklearn.decomposition import PCA

# 次元圧縮 (13次元 -> 2次元)
X_pca = PCA(n_components=2, random_state=42).fit_transform(X_std)  # n_componentsは圧縮後の次元数

X_pca

補足：主成分を直接計算

In [ ]:
import numpy as np
# 分散共分散行列の作成
cov_mat = np.cov(X_std.T)
# 分散共分散行列の固有値、固有ベクトルを作成
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
# 固有値、固有ベクトルのペアを作成
eigen_pairs = [(np.abs(eigen_vals[i]), eigen_vecs[:,i]) for i in range(len(eigen_vals))]
# 上記のペアを固有値の大きい順にソート
eigen_pairs.sort(key=lambda k: k[0], reverse=True)

w1 = eigen_pairs[0][1]  # 第1主成分に対応する固有ベクトル
w2 = eigen_pairs[1][1]  # 第2主成分に対応する固有ベクトル

# 射影行列の作成
W = np.stack([w1, w2], axis=1)
# 次元圧縮 (13次元 -> 2次元)
X_pca = X_std @ W # 行列積

X_pca

主成分（PC1、PC2）2次元空間へのプロット

In [ ]:
colors = ['#de3838', '#007bc3', '#ffd12a']  # 緋色、露草色、山吹色 
markers = ['o', 'x', ',']
for l, c, m, in zip(np.unique(y), colors, markers):
    plt.scatter(X_pca[y==l, 0], X_pca[y==l, 1],
                c=c, marker=m, label=l)

plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.legend()
plt.show()